<a href="https://colab.research.google.com/github/mcruzr0609/pastillitas/blob/main/pre_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import glob
import pandas as pd
import xml.etree.ElementTree as ET


def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                    int(root.find('size')[0].text),
                    int(root.find('size')[1].text),
                    member[0].text,
                    int(member[4][0].text),
                    int(member[4][1].text),
                    int(member[4][2].text),
                    int(member[4][3].text)
                    )
            xml_list.append(value)
    column_name = ['filename', 'width', 'height',
                'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [ ]:
import csv
import xml.etree.cElementTree as ET

def csv_to_xml(csv_path, resized_images_path, labels_path, folder):
    f = open(csv_path, 'r')
    reader = csv.reader(f)
    header = next(reader)
    old_filename = None
    for row in reader:
        filename = row[0]
        if filename == old_filename:
            object = ET.SubElement(annotation, 'object')
            ET.SubElement(object, 'name').text = row[3]
            ET.SubElement(object, 'pose').text = 'Unspecified'
            ET.SubElement(object, 'truncated').text = '0'
            ET.SubElement(object, 'difficult').text = '0'
            bndbox = ET.SubElement(object, 'bndbox')
            ET.SubElement(bndbox, 'xmin').text = row[4]
            ET.SubElement(bndbox, 'ymin').text = row[5]
            ET.SubElement(bndbox, 'xmax').text = row[6]
            ET.SubElement(bndbox, 'ymax').text = row[7]
        else:
            if old_filename is not None:
                labels_file = old_filename.replace('.jpg', '.xml')
                tree = ET.ElementTree(annotation)
                tree.write(labels_path + labels_file)

            annotation = ET.Element('annotation')
            ET.SubElement(annotation, 'folder').text = folder
            ET.SubElement(annotation, 'filename').text = filename
            ET.SubElement(annotation, 'path').text =     resized_images_path + filename
            source = ET.SubElement(annotation, 'source')
            ET.SubElement(source, 'database').text = 'Unknown'
            size = ET.SubElement(annotation, 'size')
            ET.SubElement(size, 'width').text = row[1]
            ET.SubElement(size, 'height').text = row[2]
            ET.SubElement(size, 'depth').text = '3'
            ET.SubElement(annotation, 'segmented').text = '0'

            object = ET.SubElement(annotation, 'object')
            ET.SubElement(object, 'name').text = row[3]
            ET.SubElement(object, 'pose').text = 'Unspecified'
            ET.SubElement(object, 'truncated').text = '0'
            ET.SubElement(object, 'difficult').text = '0'
            bndbox = ET.SubElement(object, 'bndbox')
            ET.SubElement(bndbox, 'xmin').text = row[4]
            ET.SubElement(bndbox, 'ymin').text = row[5]
            ET.SubElement(bndbox, 'xmax').text = row[6]
            ET.SubElement(bndbox, 'ymax').text = row[7]
        old_filename = filename
    f.close()

In [ ]:
from imgaug.augmentables.bbs import BoundingBoxesOnImage
from imgaug import augmenters as iaa

aug = iaa.SomeOf(2, [
    iaa.Affine(scale=(0.5, 1.5)),
    iaa.Affine(rotate=(-60, 60)),
    iaa.Affine(translate_percent={"x": (-0.3, 0.3), "y": (-0.3,   0.3)}),
    iaa.Fliplr(1),
    iaa.Multiply((0.5, 1.5)),
    iaa.GaussianBlur(sigma=(1.0, 3.0)),
    iaa.AdditiveGaussianNoise(scale=(0.03 * 255, 0.05 * 255)),
    iaa.Add((-25, 25)),
    iaa.MotionBlur(k=15),
    iaa.MultiplySaturation((0.5, 1.5)),
    iaa.LogContrast(gain=(0.6, 1.4)),
    iaa.Flipud(1)
])

In [ ]:
import imageio
import pandas as pd
import numpy as np
import re
import os
import glob

def image_aug(df, images_path, aug_images_path, image_prefix, augmentor):
    aug_bbs_xy = pd.DataFrame(columns=
                              ['filename','width','height','class', 'xmin', 'ymin', 'xmax', 'ymax']
                             )
    grouped = df.groupby('filename')
    
    for filename in df['filename'].unique():
        group_df = grouped.get_group(filename)
        group_df = group_df.reset_index()
        group_df = group_df.drop(['index'], axis=1)   
        image = imageio.imread(images_path+filename)
        bb_array = group_df.drop(['filename', 'width', 'height', 'class'], axis=1).values
        bbs = BoundingBoxesOnImage.from_xyxy_array(bb_array, shape=image.shape)
        image_aug, bbs_aug = augmentor(image=image, bounding_boxes=bbs)
        bbs_aug = bbs_aug.remove_out_of_image()
        bbs_aug = bbs_aug.clip_out_of_image()
        
        if re.findall('Image...', str(bbs_aug)) == ['Image([]']:
            pass

        else:
            imageio.imwrite(aug_images_path+image_prefix+filename, image_aug)  
            info_df = group_df.drop(['xmin', 'ymin', 'xmax', 'ymax'], axis=1)    
            for index, _ in info_df.iterrows():
                info_df.at[index, 'width'] = image_aug.shape[1]
                info_df.at[index, 'height'] = image_aug.shape[0]
            info_df['filename'] = info_df['filename'].apply(lambda x: image_prefix+x)
            bbs_df = bbs_obj_to_df(bbs_aug)
            aug_df = pd.concat([info_df, bbs_df], axis=1)
            aug_bbs_xy = pd.concat([aug_bbs_xy, aug_df])            
    
    aug_bbs_xy = aug_bbs_xy.reset_index()
    aug_bbs_xy = aug_bbs_xy.drop(['index'], axis=1)
    return aug_bbs_xy

In [ ]:
def bbs_obj_to_df(bbs_object):
    bbs_array = bbs_object.to_xyxy_array()
    df_bbs = pd.DataFrame(bbs_array, columns=['xmin', 'ymin', 'xmax', 'ymax'])
    return df_bbs

In [ ]:
xml_df = xml_to_csv('xmls/')
for i in range(10):
    augmented_images_df = image_aug(xml_df, 'resized_images/', 'resized_images/', 'aug{}_'.format(i), aug)
    augmented_images_df.to_csv('aug{}_images.csv'.format(i), index=False)
    csv_to_xml(csv_path='aug{}_images.csv'.format(i),
                resized_images_path='resized_images/', 
                labels_path='labels_path/',
                folder='resized_images')
    os.remove('aug{}_images.csv'.format(i))

In [ ]:
import os
import shutil
import random
#grep "<xmin />" aug* for corrupted data
images_path = 'resized_images/'
labels_path = 'labels_path/'
train_path = 'dataset/train/'
validation_path = 'dataset/validation/'


for image_file in os.listdir(images_path):
    labels_file = image_file.replace('.jpg', '.xml')
    if random.uniform(0, 1) > 0.2:
        shutil.copy(images_path + image_file, train_path + 'images/'
                    + image_file)
        shutil.copy(labels_path + labels_file, train_path +
                    'annotations/' + labels_file)
    else:
        shutil.copy(images_path + image_file, validation_path +
                    'images/' + image_file)
        shutil.copy(labels_path + labels_file, validation_path +
                    'annotations/' + labels_file)